# Automated Nuclei Identification in Divergent Images

### Rebecca Han, Jack Findley

### 04.12.18

# Roadmap
***

**Motivation**<p>

**Project Scope**<br>
&emsp; Dataset<br>
&emsp; Challenges<br>
&emsp; Workflow<p>

**Methods**<br>
&emsp; Detection<br>
&emsp; Segmentation<br>
&emsp; Classification<p>

**Results**<br>
&emsp; Error metrics<p>

**Improvement**<br>
&emsp; K-means clustering<p>

**Conclusion**<br>
&emsp; Takeaways<br>
&emsp; Future work

# Motivation: Automated Nuclei Identification
***

600,000 people in the US (0.2% of population) die from cancer every year.

40% of all men and women will be diagnosed with cancer at some point in their lives.

Early diagnosis is critical to higher treatment success.

Cancer cells display different biomarkers, visible in immunohistochemical (IHC) imaging.

Rapid and accurate image screening would tremendously improve cancer treatment.

# Project Scope
***

#### Dataset

665 training images

Each has between 4 to ~380 masks → 32145 files total

Largest image size is 1024x1536 → ~1.5 million pixels!

![Figure 1. Example of a training image with 58 true identified objects (nuclei) as individual and combined masks](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig1.png)

#### Challenges

Variation in imaging condition (illumination, contrast, fluorescence and staining)

Variation in cell / tissue type (epithelial, lung, gastrointestinal, etc)

Cell aggregation

#### Workflow

![Figure 0. Schematic of project workflow](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig0.png)

# Methods: Naive to Sophisticated
***

Detection<br>
&emsp; Normalize colour<br>
&emsp; Binarize image

Segmentation<br>
&emsp; Edge finding<br>
&emsp; Shape/size fitting - concavity, ellipse, contour

Classification, dimensionality reduction<br>
&emsp; "Machine learning" - cluster analysis, random forests, convolutional neural networks

**(1) Random forest classification (2) + K-means clustering**

# Detection: Otsu's Thresholding
***

Clustering method

Assumes pixel intensity follows bimodal, well-separated distribution

Minimize within-class variance  $\sigma_w^2 (t) = \omega_0 (t) \sigma_0^2 (t) + \omega_1 (t) \sigma_1^2 (t)$

Works well if ~same number of pixels in each class

![Figure 3. Comparison of thresholded objects and total true objects](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig3.png)

It fails dramatically when intensity is not bimodal.

![Figure 4. Original training image 177, grayscale, and thresholded](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig4.png)

It is most successful when nuclei are solid, distinct and not aggregated.

![Figure 5. Original training image 1, grayscale, and thresholded](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig5.png)

# Segmentation: Watershed
***

Find object borders in a grayscale image

Topographic map based on brightness

Flood the map from sources in each local minimum 

Build barriers when different water sources meet

![Watershed flooding in 1D](https://imagej.net/_images/thumb/c/c5/Watershed-flooding-graph.png/467px-Watershed-flooding-graph.png)

Watershed is better than thresholding, but notoriously over-segments.

![Figure 6. Training image 510 true objects, predicted objects, and markers](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig6.png)

However, watershed is a great way to reduce our feature vector.

![Figure 7. Training image 510 sure background, sure foreground, and uncertain area](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig7.png)

# (1) Random Forest Classification
***

Decision tree classifiers are supervised learning algorithms

Split the data according to some variable, at some value

Find key variable and value that minimize variance (entropy) between sets

Random forests are ensembles of trees that vote on each pixel's class (0, 1)

Trained on the uncertain area

X_train = feature vector (13,022,247 x 7)
* Grayscale pixel intensity (continuous)
* Watershed prediction for a pixel (discrete)
* Gradient, Laplacian of pixel intensity (discrete)
* Intensity of rgb colour channels (continuous)

Y_train = sum of true masks (13,022,247 x 1)

Random forest can correctly predict non-bimodal images with cell overlap.

![Figure 8. Comparison of true objects and threshold, watershed, random forest predictions for training image 177](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig8.png)

The random forest classifier performs significantly better. 

It has "learned" to split this image across a higher grayscale intensity. 

![Figure 9. Comparison of true objects and threshold, watershed, random forest predictions for training image 23](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig9.png)

Random forest also works in cases where thresholding flips objects/background.

However, it struggles on noisy or partially occluded images.

This is why leading algorithms combine contour fitting with machine learning.

![Figure 10. Comparison of true objects and threshold, watershed, random forest predictions for training image 175](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig10.png)

# Results: Quantifying Error of Different Methods
***

Multiple ways of reporting error.

(1) Confusion matrix

(2) Statistical error metrics relevant in binary classification
* Accuracy = (TP + TN) / (TP + FP + TN + FN) = (TP + TN) / (Total)
* Precision = (TP) / (TP + FP)
* Recall = (TP) / (TP + FN)
* F1 Score = 2$*$(Precision $*$ Recall) / (Precision + Recall)

(3) Jaccard index (Intersection over Union) measures similarity between sample sets<br>
For a set of predicted object pixels A and true object pixels B,
$$IoU(A,B) = \frac{A \cap B}{A \cup B}$$
<b>Calculating IoU was a bottleneck (~8.5 hours).</b>

![Figure 19. Confusion matrix of watershed and random forest classifier predictions](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig19.png)

We report error averaged across 665 training images.

Watershed may seem better, but this is due to class imbalance.

|   | Watershed|Random Forest|
|------|------|------|
|Accuracy| 95.5% | 96.0% |
|Precision| 86.6% | 80.0% |
|F1 Score| 82.3% | 80.7% |
|IoU| 58.2% | 55.3% |

# Improvements: Dimensionality Reduction 
***

We flagged images with < 70% accuracy OR precision.

The poorly predicted images all tend to be of certain type.

Intuitively, dimensionality reduction should work well here.

Why did we not start with dimensionality reduction?

#### We did, but dimensionality reduction takes a long time.

Training linear SVM took > 10 times longer than training the random forest.

Based on complexity analysis, k-means clustering was most efficient.

|   | Complexity (of training) | n (# of samples) | d (# of features) |
|------|------|------|------|
|PCA | $O(n*d^2$+$d^3)$ | 665 images | ~1.5 M pixels in largest image | 
|SVM (RBF kernel)| $O(n^3)$ [1] | ~13 M reduced pixels | 7 features |
|SVM (Linear) | $O(n^2)$ [2] | ~13 M reduced pixels | 7 features |
|KMEANS (Lloyd's algorithm)| $O(n*d*$(6 clusters)$*$(300 iterations)$)$ | 665 images | 6 geometry, 3 colour features |
|Random Forest |  $O(n*d*$(10 trees)$*$(5 depth)$)$ | ~13 M reduced pixels | 7 features |

[1] Srebro and Shalev-Shwartz. (**2008**) *Proceedings of ICML*, doi:10.1.1.139.2112

[2] Bordes, et al. (**2005**) *J Mach Learn Res*, 6, 1579–1619, doi:10.1.1.60.9676.

# (2) Random Forest + K-means Clustering
***

By just looking at the training set, we guess 6 clusters of images.

Images in the training set differ based on cell features and colour.

Geometry feature vector (6):
* average grayscale intensity
* image size (width, length)
* contour area (max, mean)
* number of contours per image

Colour feature vector (3):
* average red, green, blue intensity

Watershed does great on Class 0 images, hence why it seems to outperform the random forest.

![Figure 20. Histogram of Training Images, Separated by K-means Class](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig20.png)

#### Retraining on each cluster yields significant improvement

Error averaged across all 665 images.

The cluster specific random forest performs better than either method.

|   | Watershed|Random Forest|**Cluster Specific RF**|
|------|------|------|------|
|Accuracy| 95.5% | 96.0% | **96.9%** |
|Precision| 86.6% | 80.0% | **87.0%** |
|F1 Score| 82.3% | 80.7% | **85.9%** |
|IoU| 58.2% | 55.3% | **65.9%** |

|   | Watershed|Random Forest|**Cluster Specific RF**|
|------|------|------|------|
|Accuracy| 96.7% | 96.9% | **97.2%** |
|Precision| 90.1% | 79.9% | **81.2%** |
|F1 Score| 85.0% | 81.1% | **84.3%** |

![Figure 13. Example Images Representative of K-means Class 0](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig13.png)

|   | Watershed|Random Forest|**Cluster Specific RF**|
|------|------|------|------|
|Accuracy| 94.7% | 95.0% | **95.4%** |
|Precision| 84.9% | 87.4% | **86.1%** |
|F1 Score| 84.3% | 83.2% | **85.8%** |

![Figure 14. Example Images Representative of K-means Class 1](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig14.png)

|   | Watershed|Random Forest|**Cluster Specific RF**|
|------|------|------|------|
|Accuracy| 91.1% | 92.3% | **93.2%** |
|Precision| 79.6% | 78.2% | **81.9%** |
|F1 Score| 78.3% | 80.1% | **80.7%** |

![Figure 15. Example Images Representative of K-means Class 2](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig15.png)

|   | Watershed|Random Forest|**Cluster Specific RF**|
|------|------|------|------|
|Accuracy| 84.2% | 83.1% | **97.4%** |
|Precision| 36.5% | 38.8% | **80.5%** |
|F1 Score| 48.5% | 42.8% | **76.2%** |

![Figure 16. Example Images Representative of K-means Class 3](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig16.png)

|   | Watershed|Random Forest|**Cluster Specific RF**|
|------|------|------|------|
|Accuracy| 80.6% | 97.7% | **98.9%** |
|Precision| 8.7% | 48.5% | **72.2%** |
|F1 Score| 16.0% | 59.8% | **69.0%** |

![Figure 17. Example Images Representative of K-means Class 4](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig17.png)

|   | Watershed|Random Forest|**Cluster Specific RF**|
|------|------|------|------|
|Accuracy| 96.2% | 96.1% | **96.7%** |
|Precision| 92.9% | 88.8% | **89.4%** |
|F1 Score| 86.3% | 86.2% | **88.6%** |

![Figure 18. Example Images Representative of K-means Class 5](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig18.png)

![Figure 11. Comparison of true objects, watershed, random forest, and cluster-specific random forest predictions for training image 23](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig11.png)

![Figure 12. Comparison of true objects, watershed, random forest, and cluster-specific random forest predictions for training image 175](https://raw.githubusercontent.com/rbhan/chbe8803-nuclei-identification/master/images/fig12.png)

# Conclusion
***

#### Takeaways

Models typically over-predict nucleus size (many false positives)

No "one-size-fits-all" model for all cell type/imaging conditions

Image analysis gets very costly very fast, so pay attention to algorithm complexity

This is a hard problem, with a lot of money in it ($100k prize)

#### Future Work

Improving class balance - train on more images (IHC data is readily available)

Improving algorithms - add iterative PCA, neural networks, shape/size fitting

Improving runtime - parallelize, run on a cluster (PACE has weird packages)

# We are happy to take questions.